In [25]:
import pandas as pd
import altair as alt


df = pd.read_csv('merged_mortgage_data.csv')
df['observation_date'] = pd.to_datetime(df['observation_date'])


df_filtered = df[df['mbs_z_score'].notna() & df['delinq_z_score'].notna()].copy()


df_standardized = df_filtered[['observation_date', 'year', 'mbs_z_score', 'delinq_z_score']].melt(
    id_vars=['observation_date', 'year'],  
    value_vars=['mbs_z_score', 'delinq_z_score'],
    var_name='metric',
    value_name='z_score'
)

overall_chart = alt.Chart(df_standardized).mark_line(size=3).encode(
    x=alt.X('observation_date:T', title='Date'),
    y=alt.Y('z_score:Q', 
            title='Z-score (vs entire 2009-2025 period)',
            scale=alt.Scale(domain=[-2, 2])),
    color=alt.Color('metric:N', 
                    scale=alt.Scale(domain=['mbs_z_score', 'delinq_z_score'],
                                   range=['steelblue', 'orange']),
                    legend=alt.Legend(title='Metric',
                                     labelExpr="datum.value == 'mbs_z_score' ? 'MBS Price' : 'Delinquency Rate'")),
    tooltip=[
        alt.Tooltip('observation_date:T', title='Date', format='%Y-%m-%d'),
        alt.Tooltip('year:O', title='Year'),
        alt.Tooltip('metric:N', title='Metric'),
        alt.Tooltip('z_score:Q', title='Z-score', format='.2f')
    ]
).properties(
    width=900,
    height=350,
    title='Z Scores of Mortgage Backed Securities Prices and Deliquency Rates Plotted Over Time'
)

zero_line1 = alt.Chart(pd.DataFrame({'y': [0]})).mark_rule(
    color='gray', strokeDash=[10, 10], size=2
).encode(y='y:Q')

overall_with_line = overall_chart + zero_line1


yearly_avg = df_filtered.groupby('year').agg({
    'mbs_z_score': 'mean',
    'delinq_z_score': 'mean'
}).reset_index()

yearly_melted = yearly_avg.melt(
    id_vars='year',
    value_vars=['mbs_z_score', 'delinq_z_score'],
    var_name='metric',
    value_name='z_score'
)

yearly_bars = alt.Chart(yearly_melted).mark_bar().encode(
    x=alt.X('year:O', title='Year', axis=alt.Axis(labelAngle=-45)),
    y=alt.Y('z_score:Q', 
            title='Average Z-score (vs all years)',
            scale=alt.Scale(domain=[-2, 2])),
    color=alt.Color('metric:N',
                    scale=alt.Scale(domain=['mbs_z_score', 'delinq_z_score'],
                                   range=['steelblue', 'orange']),
                    legend=alt.Legend(title='Metric',
                                     labelExpr="datum.value == 'mbs_z_score' ? 'MBS Price' : 'Delinquency Rate'")),
    xOffset='metric:N',
    tooltip=[
        alt.Tooltip('year:O', title='Year'),
        alt.Tooltip('metric:N', title='Metric'),
        alt.Tooltip('z_score:Q', title='Avg Z-score', format='.2f')
    ]
).properties(
    width=900,
    height=350,
    title='Year-by-Year Risk Profile: How do the Price of Mortgage Backed Securities Z scores Compare to the Relative Deliquency Rates Z Scores Through Years?'
)

zero_line2 = alt.Chart(pd.DataFrame({'y': [0]})).mark_rule(
    color='gray', strokeDash=[10, 10], size=2
).encode(y='y:Q')

yearly_with_line = yearly_bars + zero_line2

combined = alt.vconcat(overall_with_line, yearly_with_line)
combined.save('mbs_risk_analysis.html')